# Team SML

In [29]:
import pandas as pd
import geopandas as gpd
import geopy.distance
import math
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression


In [5]:


url = 'https://raw.githubusercontent.com/cividi/st-gallen-urban-indicators/main/data/price-monitoring/price-monitoring-extended.csv'
df = pd.read_csv(url, index_col=0)
print(df.head(5))

mycols = ['rent','lat', 'lng', 'LR_DAY', 'LR_NIGHT']

pipe = Pipeline([
('selector', ("selector", ColumnTransformer([
        ("selector", "passthrough", mycols)
    ], remainder="drop"))),
    ('regression', LinearRegression())
])

pipeline.fit(X=df.iloc[:, 0:5], y=df['rent'])

                                                   object_ref category  \
id                                                                       
108761792                                         hgok2876852     rent   
2147994348  44.53.3003.499622.e75b7976-c57b-11e9-9cb0-a4bf...     rent   
3000166218                                    21301.01.420030     rent   
3000251780                                        hgoe3023792     rent   
3000402506                                      03159.01.0132     rent   

                           date  flatType floor  rent  rent_add  rent_net  \
id                                                                          
108761792   2021-10-12 19:06:18       NaN     4  1745     200.0    1545.0   
2147994348  2021-10-12 19:08:41       NaN     3  1720       NaN       NaN   
3000166218  2021-10-12 20:47:01       NaN     2  1150       NaN       NaN   
3000251780  2021-10-13 07:27:31       NaN     2  1390       NaN       NaN   
3000402506  2021-10

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' '('selector', ColumnTransformer(transformers=[('selector', 'passthrough',
                                 ['rent', 'lat', 'lng', 'LR_DAY', 'LR_NIGHT'])]))' (type <class 'tuple'>) doesn't

In [92]:
urlNew = 'https://raw.githubusercontent.com/cividi/st-gallen-urban-indicators/main/data/price-monitoring/price-monitoring-extended.csv'

df_rentalobjects = pd.read_csv(urlNew, index_col=0)
df_supermarkets = gpd.read_file('./mapdata/supermarkets.geojson')
df_rentalobjects['supermarketsnearby'] = 0
df_rentalobjects = df_rentalobjects.reset_index()
df_supermarkets = df_supermarkets.reset_index()

for index, row_rentalobject in df_rentalobjects.reset_index().iterrows():
    count_supermarkets_nearby = 0
    if not math.isnan(row_rentalobject['lng']) and not math.isnan(row_rentalobject['lat']):
        coords_rentalobject = (row_rentalobject['lng'], row_rentalobject['lat'])
        for index2, row_supermarket in df_supermarkets.iterrows():
            geometry_supermarket = row_supermarket['geometry']
            coords_supermarket = (geometry_supermarket.x, geometry_supermarket.y)
            if not math.isnan(geometry_supermarket.x) and not math.isnan(geometry_supermarket.y):
                if geopy.distance.distance(coords_rentalobject, coords_supermarket).m <= 500:
                    count_supermarkets_nearby = count_supermarkets_nearby + 1
    df_rentalobjects.at[index, 'supermarketsnearby'] = count_supermarkets_nearby
df_rentalobjects.to_csv("./data/price-monitoring-calculated.csv", index=False)

        
           
    
        
        



